In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!pip install --upgrade torch torchvision torchaudio
!pip install --upgrade ultralytics


# 데이터 전처리

데이터셋에 바운딩박스와 폴리곤 객체이터셋에 함께 있어 폴리곤 -> 바운딩박스 최소 외접 사격으로 변환

In [ ]:

# 폴리곤 데이터 박스 데이터로 변환

import os
import numpy as np
from tqdm import tqdm
import shutil

base_dir = '/content/drive/MyDrive/final year car paint defect.v1i.yolov8'
output_dir = base_dir + '_bbox'  # 변환된 데이터셋 저장 경로

sets = ['train', 'valid', 'test']

def convert_polygon_to_bbox_line(line):
    nums = list(map(float, line.strip().split()))
    if len(nums) == 5:
        return line.strip()  # 이미 bbox 형식이면 그대로 사용
    cls = int(nums[0])
    coords = np.array(nums[1:]).reshape(-1, 2)
    xmin, ymin = coords.min(axis=0)
    xmax, ymax = coords.max(axis=0)
    cx, cy = (xmin + xmax) / 2, (ymin + ymax) / 2
    w, h = xmax - xmin, ymax - ymin
    return f"{cls} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}"

for split in sets:
    label_dir = os.path.join(base_dir, split, 'labels')
    img_dir = os.path.join(base_dir, split, 'images')

    out_label_dir = os.path.join(output_dir, split, 'labels')
    out_img_dir = os.path.join(output_dir, split, 'images')
    os.makedirs(out_label_dir, exist_ok=True)
    os.makedirs(out_img_dir, exist_ok=True)

    print(f" {split} 변환 중...")

    for fname in tqdm(os.listdir(label_dir)):
        src_label_path = os.path.join(label_dir, fname)
        dst_label_path = os.path.join(out_label_dir, fname)

        # Polygon 라벨 → Bounding Box
        with open(src_label_path, 'r') as f:
            lines = f.readlines()
        converted_lines = [convert_polygon_to_bbox_line(line) for line in lines]

        with open(dst_label_path, 'w') as f:
            f.write('\n'.join(converted_lines))

        # 이미지도 복사
        img_name = fname.replace('.txt', '.jpg')
        src_img_path = os.path.join(img_dir, img_name)
        dst_img_path = os.path.join(out_img_dir, img_name)
        if os.path.exists(src_img_path):
            shutil.copyfile(src_img_path, dst_img_path)

print(" Polygon → BBox 변환 완료!")
print(f" 결과 경로: {output_dir}")
